<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/wireless-pr3d/sdr5g_latency_measurement_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [1]:
import os, re
from getpass import getpass

with open('/content/sdr-test-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [2]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.7/487.7 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Import packages

In [3]:
import json, time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, restart_sdr

In the next cell, we reserve the required equipment and resources to form an end to end sdr 5g setup. We reserve 2 SDRs and 3 workers.

In [18]:
# reserve sdr-02 SFP
sdr02_lease = show_reservation_byname("sdr-02-lease")
if not sdr02_lease:
    sdr02_lease = reserve(
        { "type":"network", "name": "sdr-02", "net_name": "sdr-02", "segment_id": "104", "duration": { "days":7, "hours":0 } }
    )

# reserve sdr-03 SFP
sdr03_lease = show_reservation_byname("sdr-03-lease")
if not sdr03_lease:
    sdr03_lease = reserve(
        { "type":"network", "name": "sdr-03", "net_name": "sdr-03", "segment_id": "106", "duration": { "days":7, "hours":0 } }
    )

# reserve sdr-08 SFP
sdr08_lease = show_reservation_byname("sdr-08-lease")
if not sdr08_lease:
    sdr08_lease = reserve(
        { "type":"network", "name": "sdr-08", "net_name": "sdr-08", "segment_id": "116", "duration": { "days":7, "hours":0 } }
    )

# reserve worker-01
worker01_lease = show_reservation_byname("worker-01-lease")
if not worker01_lease:
    worker01_lease = reserve(
        { "type":"device", "name":"worker-01", "duration": { "days":7, "hours":0 } }
    )
worker01_reservation_id = worker01_lease["reservations"][0]["id"]

# reserve worker-02
worker02_lease = show_reservation_byname("worker-02-lease")
if not worker02_lease:
    worker02_lease = reserve(
        { "type":"device", "name":"worker-02", "duration": { "days":7, "hours":0 } }
    )
worker02_reservation_id = worker02_lease["reservations"][0]["id"]

# reserve worker-03
worker03_lease = show_reservation_byname("worker-03-lease")
if not worker03_lease:
    worker03_lease = reserve(
        { "type":"device", "name":"worker-03", "duration": { "days":7, "hours":0 } }
    )
worker03_reservation_id = worker03_lease["reservations"][0]["id"]

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2023-08-06 22:39:11.087 | INFO     | chi.expeca:reserve:167 - reserving sdr-02
2023-08-06 22:39:13.754 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for sdr-02-lease with id 190074bf-d532-4ca4-8386-392a02f4359b to become "ACTIVE"
2023-08-06 22:39:19.025 | INFO     | chi.expeca:wait_until_lease_status:69 - lease sdr-02-lease with id 190074bf-d532-4ca4-8386-392a02f4359b is PENDING.
2023-08-06 22:39:24.306 | INFO     | chi.expeca:wait_until_lease_status:69 - lease sdr-02-lease with id 190074bf-d532-4ca4-8386-392a02f4359b is PENDING.
2023-08-06 22:39:29.568 | INFO     | chi.expeca:wait_until_lease_status:69 - lease sdr-02-lease with id 190074bf-d532-4ca4-8386-392a02f4359b is PENDING.
2023-08-06 22:39:34.850 | INFO     | chi.expeca:wait_until_lease_status:69 - lease sdr-02-lease with id 190074bf-d532-4ca4-8386-392a02f4359b is PENDING.
2023-08-06 22:39:40.122 | INFO     | chi.expeca:wait_until_lease_status:69 - lease sdr-02-lease with id 190074bf-d532-4ca4-8386-392

[
    {
        "name": "sdr-03-lease",
        "id": "4793fee5-4a64-4fd6-9186-7e019d821ce3",
        "reservation_id": "df5ac2d2-bb8f-4cf0-a40d-086ec8fe3984",
        "status": "ACTIVE",
        "end_date": "2023-08-13T22:40:00.000000"
    },
    {
        "name": "sdr-08-lease",
        "id": "9a2920b1-8b4a-4cd6-9af2-ed71548b98b1",
        "reservation_id": "c20fe26a-8b65-45a9-bd1b-8d3118ace176",
        "status": "ACTIVE",
        "end_date": "2023-08-13T22:41:00.000000"
    },
    {
        "name": "sdr-02-lease",
        "id": "190074bf-d532-4ca4-8386-392a02f4359b",
        "reservation_id": "841b7132-9c03-4eaf-9fba-c45a134a59cc",
        "status": "ACTIVE",
        "end_date": "2023-08-13T22:39:00.000000"
    },
    {
        "name": "worker-01-lease",
        "id": "8857b02b-e752-4d04-8876-2014006eab67",
        "reservation_id": "3c57a14b-1ee8-4441-9fc0-ad24751a581d",
        "status": "ACTIVE",
        "end_date": "2023-08-13T22:42:00.000000"
    },
    {
        "name": "work

In the following section we setup the 5g core (refer to [here](https://github.com/KTH-EXPECA/oai5g-docker/blob/main/docs/5g-sa-chi-tutorial.md) for more info).
It creates:

1.   5G core network with subnet "192.168.70.128/26"
2.   MySQL, UDR, UDM, AUSF, AMF, SMF, and SPGWU services on worker-01, on interface ens1f1.

In [21]:
# create or get 5g core's network
corenet = None
try:
    corenet = chi.network.get_network("5gcn-net")
except Exception as ex:
    logger.info(f"could not find 5gcn-net, creating it...")

if not corenet:
    corenet = chi.network.create_network("5gcn-net")
    chi.network.create_subnet("5gcn-net-subnet",corenet["id"],"192.168.70.128/26",enable_dhcp=False)
    logger.success("5gcn-net is created.")

# run MySQL
status = get_container_status("5gcn-1-mysql")
if not status:
    chi.container.create_container(
        name = "5gcn-1-mysql",
        image = "samiemostafavi/expeca-mysql",
        reservation_id = worker01_reservation_id,
        nets = [{ "network" : corenet['id'] }],
        labels = {"networks.1.interface":"ens1f1","networks.1.ip":"192.168.70.131/26"},
    )
    chi.container.wait_for_active("5gcn-1-mysql")
logger.success("5gcn-1-mysql is up.")

# run UDR
status = get_container_status("5gcn-2-udr")
if not status:
    chi.container.create_container(
        name = "5gcn-2-udr",
        image = "samiemostafavi/expeca-udr",
        reservation_id = worker01_reservation_id,
        environment = {"UDR_INTERFACE_NAME_FOR_NUDR":"net1","USE_FQDN_DNS":"no","REGISTER_NRF":"no"},
        nets = [{ "network" : corenet['id'] }],
        labels = {"networks.1.interface":"ens1f1","networks.1.ip":"192.168.70.136/26"},
    )
    chi.container.wait_for_active("5gcn-2-udr")
logger.success("5gcn-2-udr is up.")

# run UDM
status = get_container_status("5gcn-3-udm")
if not status:
    chi.container.create_container(
        name = "5gcn-3-udm",
        image = "samiemostafavi/expeca-udm",
        reservation_id = worker01_reservation_id,
        environment = {"SBI_IF_NAME":"net1","USE_FQDN_DNS":"no","REGISTER_NRF":"no"},
        nets = [{ "network" : corenet['id'] }],
        labels = {"networks.1.interface":"ens1f1","networks.1.ip":"192.168.70.137/26"},
    )
    chi.container.wait_for_active("5gcn-3-udm")
logger.success("5gcn-3-udm is up.")

# run AUSF
status = get_container_status("5gcn-4-ausf")
if not status:
    chi.container.create_container(
        name = "5gcn-4-ausf",
        image = "samiemostafavi/expeca-ausf",
        reservation_id = worker01_reservation_id,
        environment = {"SBI_IF_NAME":"net1","USE_FQDN_DNS":"no","REGISTER_NRF":"no"},
        nets = [{ "network" : corenet['id'] }],
        labels = {"networks.1.interface":"ens1f1","networks.1.ip":"192.168.70.138/26"},
    )
    chi.container.wait_for_active("5gcn-4-ausf")
logger.success("5gcn-4-ausf is up.")

# run AMF
status = get_container_status("5gcn-5-amf")
if not status:
    chi.container.create_container(
        name = "5gcn-5-amf",
        image = "samiemostafavi/expeca-amf",
        reservation_id = worker01_reservation_id,
        environment = {"AMF_INTERFACE_NAME_FOR_NGAP":"net1","AMF_INTERFACE_NAME_FOR_N11":"net1","USE_FQDN_DNS":"no","NF_REGISTRATION":"no","SMF_SELECTION":"no"},
        nets = [{ "network" : corenet['id'] }],
        labels = {"networks.1.interface":"ens1f1","networks.1.ip":"192.168.70.132/26"},
    )
    chi.container.wait_for_active("5gcn-5-amf")
logger.success("5gcn-5-amf is up.")

# run SMF
status = get_container_status("5gcn-6-smf")
if not status:
    chi.container.create_container(
        name = "5gcn-6-smf",
        image = "samiemostafavi/expeca-smf",
        reservation_id = worker01_reservation_id,
        environment = {"USE_FQDN_DNS":"no","SMF_INTERFACE_NAME_FOR_N4":"net1","SMF_INTERFACE_NAME_FOR_SBI":"net1","REGISTER_NRF":"no","DISCOVER_UPF":"no"},
        nets = [{ "network" : corenet['id'] }],
        labels = {"networks.1.interface":"ens1f1","networks.1.ip":"192.168.70.133/26"},
    )
    chi.container.wait_for_active("5gcn-6-smf")
logger.success("5gcn-6-smf is up.")


# run SPGWU
status = get_container_status("5gcn-7-spgwu")
if not status:
    chi.container.create_container(
        name = "5gcn-7-spgwu",
        image = "samiemostafavi/expeca-spgwu",
        reservation_id = worker01_reservation_id,
        environment = {"SGW_INTERFACE_NAME_FOR_S1U_S12_S4_UP":"net1","SGW_INTERFACE_NAME_FOR_SX":"net1","PGW_INTERFACE_NAME_FOR_SGI":"net1","USE_FQDN_NRF":"no","REGISTER_NRF":"no"},
        nets = [{ "network" : corenet['id'] }],
        labels = {
            "networks.1.interface":"ens1f1",
            "networks.1.ip":"192.168.70.134/26",
            "capabilities.privileged":"true",
            "capabilities.add.1":"NET_ADMIN",
            "capabilities.add.2":"SYS_ADMIN",
            "capabilities.drop.1":"ALL"
        },
    )
    chi.container.wait_for_active("5gcn-7-spgwu")
logger.success("5gcn-7-spgwu is up.")

2023-08-06 22:52:30.167 | SUCCESS  | __main__:<cell line: 24>:24 - 5gcn-1-mysql is up.
2023-08-06 22:52:30.497 | SUCCESS  | __main__:<cell line: 38>:38 - 5gcn-2-udr is up.
2023-08-06 22:52:30.796 | SUCCESS  | __main__:<cell line: 52>:52 - 5gcn-3-udm is up.
2023-08-06 22:52:31.090 | SUCCESS  | __main__:<cell line: 66>:66 - 5gcn-4-ausf is up.
2023-08-06 22:52:31.389 | SUCCESS  | __main__:<cell line: 80>:80 - 5gcn-5-amf is up.
2023-08-06 22:52:45.768 | SUCCESS  | __main__:<cell line: 94>:94 - 5gcn-6-smf is up.
2023-08-06 22:52:59.956 | SUCCESS  | __main__:<cell line: 116>:116 - 5gcn-7-spgwu is up.


Successful connection configs:

*   SDR-02 (GNB)  <--->  SDR-08 (UE)
  *   GNB: "ATT_TX":"0"
  *   GNB: "ATT_RX":"0"
  *   GNB: "MAX_RXGAIN":"114"
  *   UE: e320_ue_rx_gain: 70
  *   UE: e320_ue_tx_gain: 80


In the following cell we run gnodeb on worker-02 using sdr-02.

In [ ]:
# restart SDRs to get clean

# reset SDR-02 using worker-02
sdrname = "sdr-02"
sdrnet = chi.network.get_network(sdrname+"-net")
sdrip = "10.30.1.4"
restart_sdr(sdrname,sdrnet['id'],worker02_reservation_id,"eno12409np1")

# wait 10 seconds
time.sleep(10)

# reset SDR-08 using worker-02
sdrname = "sdr-08"
sdrnet = chi.network.get_network(sdrname+"-net")
sdrip = "10.30.1.16"
restart_sdr(sdrname,sdrnet['id'],worker02_reservation_id,"eno12409np1")


In [28]:
# find sdr network
sdrname = "sdr-02"
sdrnet = chi.network.get_network(sdrname+"-net")
sdrip = "10.30.1.4"

# run 5g-gnodeb-1 on worker-02
# NOTE: do not set the ip to 10.30.1.100
chi.container.create_container(
    name = f"5g-gnodeb-01",
    image = "samiemostafavi/expeca-oai-gnb",
    reservation_id = worker02_reservation_id,
    environment = {
        "GNB_ID":"e00","GNB_NAME":"gNB-OAI","MCC":"001","MNC":"01","MNC_LENGTH":"2","TAC":"1","NSSAI_SST":"1","NSSAI_SD":"1",
        "AMF_IP_ADDRESS":"192.168.70.132","GNB_NGA_IF_NAME":"net1","GNB_NGA_IP_ADDRESS":"192.168.70.139","GNB_NGU_IF_NAME":"net1","GNB_NGU_IP_ADDRESS":"192.168.70.139",
        "USE_SA_TDD_MONO_E320":"yes",
        "ATT_TX":"0",
        "ATT_RX":"0",
        "MAX_RXGAIN":"114",
        "SDR_ADDRS":f"addr={sdrip}",
        "THREAD_PARALLEL_CONFIG":"PARALLEL_SINGLE_THREAD",
        "USE_ADDITIONAL_OPTIONS":"--sa --usrp-tx-thread-config 1 -E --gNBs.[0].min_rxtxtime 6 --RUs.[0].max_pdschReferenceSignalPower -5"
    },
    nets = [
        { "network" : corenet['id']  },
        { "network" : sdrnet['id'] },
    ],
    #entrypoint = ["sleep", "infinity"],
    labels = {
        "networks.1.interface":"ens1f1",
        "networks.1.ip":"192.168.70.139/26",
        "networks.2.interface":"eno12409np1",
        "networks.2.ip":f"10.30.1.200/24",
        "capabilities.privileged":"true",
        "resources.limits.memory":"32000Mi",
        "resources.limits.cpu":"15",
        "resources.requests.memory":"32000Mi",
        "resources.requests.cpu":"15"
    },
)
chi.container.wait_for_active(f"5g-gnodeb-01")
logger.success(f"created 5g-gnodeb-01 container.")

2023-08-06 23:26:53.559 | SUCCESS  | __main__:<cell line: 44>:44 - created 5g-gnodeb-01 container.


Run nrUE on worker-03 using sdr-08.

In [30]:
# find sdr network
sdrname = "sdr-08"
sdrnet = chi.network.get_network(sdrname+"-net")
sdrip = "10.30.1.16"

# NOTE: do not set the ip to 10.30.1.100
e320_ue_rx_gain = 70 # max 71?
e320_ue_tx_gain = 80 # max 89.75
chi.container.create_container(
    name = f"5g-nrue-01",
    image = "samiemostafavi/expeca-oai-nr-ue",
    reservation_id = worker03_reservation_id,
    environment = {
      "FULL_IMSI":"001010000000001",
      "FULL_KEY":"fec86ba6eb707ed08905757b1bb44b8f",
      "OPC":"C42449363BBAD02B66D16BC975D77CC1",
      "DNN":"oai",
      "NSSAI_SST":"1",
      "USE_ADDITIONAL_OPTIONS": "-r 106 --numerology 1 --band 78 -C 3619200000 --nokrnmod --sa -E --uicc0.imsi 001010000000001"\
                                f"--uicc0.nssai_sd 1 --usrp-args addr={sdrip} --ue-fo-compensation --ue-rxgain {e320_ue_rx_gain+44} --ue-txgain {89.75-e320_ue_tx_gain} --ue-max-power 0"
    },
    nets = [
        { "network" : sdrnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12409np1",
        "networks.1.ip":f"10.30.1.200/24",
        "capabilities.privileged":"true",
        "resources.limits.memory":"32000Mi",
        "resources.limits.cpu":"15",
        "resources.requests.memory":"32000Mi",
        "resources.requests.cpu":"15",
    },
)
chi.container.wait_for_active(f"5g-nrue-01")
logger.success(f"created 5g-nrue-01 container.")

2023-08-06 23:30:25.590 | SUCCESS  | __main__:<cell line: 42>:42 - created 5g-nrue-01 container.


CAUTION: In this cell we tear down all the configurations and release the reserved resources. The project will be clean afterwards.

In [32]:
# Remove everything and clean up

status = get_container_status("5g-nrue-01")
if status:
    chi.container.destroy_container("5g-nrue-01")
    wait_until_container_removed("5g-nrue-01")

status = get_container_status("5g-gnodeb-01")
if status:
    chi.container.destroy_container("5g-gnodeb-01")
    wait_until_container_removed("5g-gnodeb-01")

status = get_container_status("5gcn-7-spgwu")
if status:
    chi.container.destroy_container("5gcn-7-spgwu")
    wait_until_container_removed("5gcn-7-spgwu")

status = get_container_status("5gcn-6-smf")
if status:
    chi.container.destroy_container("5gcn-6-smf")
    wait_until_container_removed("5gcn-6-smf")

status = get_container_status("5gcn-5-amf")
if status:
    chi.container.destroy_container("5gcn-5-amf")
    wait_until_container_removed("5gcn-5-amf")

status = get_container_status("5gcn-4-ausf")
if status:
    chi.container.destroy_container("5gcn-4-ausf")
    wait_until_container_removed("5gcn-4-ausf")

status = get_container_status("5gcn-3-udm")
if status:
    chi.container.destroy_container("5gcn-3-udm")
    wait_until_container_removed("5gcn-3-udm")

status = get_container_status("5gcn-2-udr")
if status:
    chi.container.destroy_container("5gcn-2-udr")
    wait_until_container_removed("5gcn-2-udr")

status = get_container_status("5gcn-1-mysql")
if status:
    chi.container.destroy_container("5gcn-1-mysql")
    wait_until_container_removed("5gcn-1-mysql")

logger.info(f"stopped and removed all containers")

corenet = None
try:
    corenet = chi.network.get_network("5gcn-net")
except Exception as ex:
    logger.info(f"could not find 5gcn-net.")

if corenet:
    chi.network.delete_network(corenet["id"])
    logger.success(f"deleted the 5gcn-net")

# remove the leases
unreserve_byid(sdr02_lease["id"])
unreserve_byid(sdr03_lease["id"])
unreserve_byid(sdr08_lease["id"])
unreserve_byid(worker01_lease["id"])
unreserve_byid(worker02_lease["id"])
unreserve_byid(worker03_lease["id"])

2023-08-06 23:47:01.968 | INFO     | __main__:<cell line: 46>:46 - stopped and removed all containers
2023-08-06 23:47:09.064 | SUCCESS  | __main__:<cell line: 54>:56 - deleted the 5gcn-net
2023-08-06 23:47:09.912 | INFO     | chi.expeca:remove_lease:84 - Removing sdr-02-lease reservation with id 190074bf-d532-4ca4-8386-392a02f4359b.
2023-08-06 23:47:24.593 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for sdr-02-lease with id 190074bf-d532-4ca4-8386-392a02f4359b to become "None"
2023-08-06 23:47:29.869 | INFO     | chi.expeca:wait_until_lease_status:69 - lease sdr-02-lease with id 190074bf-d532-4ca4-8386-392a02f4359b is None.
2023-08-06 23:47:29.871 | SUCCESS  | chi.expeca:try_to_remove:95 - done
2023-08-06 23:47:30.267 | INFO     | chi.expeca:remove_lease:84 - Removing sdr-03-lease reservation with id 4793fee5-4a64-4fd6-9186-7e019d821ce3.
2023-08-06 23:47:44.323 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for sdr-03-lease with i